.. _kddcup99_dataset:

Kddcup 99 dataset
-----------------

The KDD Cup '99 dataset was created by processing the tcpdump portions
of the 1998 DARPA Intrusion Detection System (IDS) Evaluation dataset,
created by MIT Lincoln Lab [2]_. The artificial data (described on the `dataset's
homepage <https://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html>`_) was
generated using a closed network and hand-injected attacks to produce a
large number of different types of attack with normal activity in the
background. As the initial goal was to produce a large training set for
supervised learning algorithms, there is a large proportion (80.1%) of
abnormal data which is unrealistic in real world, and inappropriate for
unsupervised anomaly detection which aims at detecting 'abnormal' data, i.e.:

* qualitatively different from normal data
* in large minority among the observations.

We thus transform the KDD Data set into two different data sets: SA and SF.

* SA is obtained by simply selecting all the normal data, and a small
  proportion of abnormal data to gives an anomaly proportion of 1%.

* SF is obtained as in [3]_
  by simply picking up the data whose attribute logged_in is positive, thus
  focusing on the intrusion attack, which gives a proportion of 0.3% of
  attack.

* http and smtp are two subsets of SF corresponding with third feature
  equal to 'http' (resp. to 'smtp').

General KDD structure :

    ================      ==========================================
    Samples total         4898431
    Dimensionality        41
    Features              discrete (int) or continuous (float)
    Targets               str, 'normal.' or name of the anomaly type
    ================      ==========================================

    SA structure :

    ================      ==========================================
    Samples total         976158
    Dimensionality        41
    Features              discrete (int) or continuous (float)
    Targets               str, 'normal.' or name of the anomaly type
    ================      ==========================================

    SF structure :

    ================      ==========================================
    Samples total         699691
    Dimensionality        4
    Features              discrete (int) or continuous (float)
    Targets               str, 'normal.' or name of the anomaly type
    ================      ==========================================

    http structure :

    ================      ==========================================
    Samples total         619052
    Dimensionality        3
    Features              discrete (int) or continuous (float)
    Targets               str, 'normal.' or name of the anomaly type
    ================      ==========================================

    smtp structure :

    ================      ==========================================
    Samples total         95373
    Dimensionality        3
    Features              discrete (int) or continuous (float)
    Targets               str, 'normal.' or name of the anomaly type
    ================      ==========================================

:func:`sklearn.datasets.fetch_kddcup99` will load the kddcup99 dataset; it
returns a dictionary-like object with the feature matrix in the ``data`` member
and the target values in ``target``. The "as_frame" optional argument converts
``data`` into a pandas DataFrame and ``target`` into a pandas Series. The
dataset will be downloaded from the web if necessary.

.. topic:: References

    .. [2] Analysis and Results of the 1999 DARPA Off-Line Intrusion
           Detection Evaluation, Richard Lippmann, Joshua W. Haines,
           David J. Fried, Jonathan Korba, Kumar Das.

    .. [3] K. Yamanishi, J.-I. Takeuchi, G. Williams, and P. Milne. Online
           unsupervised outlier detection using finite mixtures with
           discounting learning algorithms. In Proceedings of the sixth
           ACM SIGKDD international conference on Knowledge discovery
           and data mining, pages 320-324. ACM Press, 2000.


In [1]:
import numpy as np
import pandas as pd
import sklearn.model_selection as model
import matplotlib.pyplot as plt
import os
from collections import Counter
import sklearn
print(sklearn.__version__)

0.23.2


In [2]:
from tensorflow.keras.utils import get_file
path_gz = get_file('kddcup.data.gz', origin='http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz')
path_txt = get_file('kddcup.names.txt', origin='http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names')

8192/1307 [============================================================================================================================================================================================] - 0s 0us/step


In [3]:
df = pd.read_csv(path_gz,header=None)

In [ ]:
df

In [4]:
with open(path_txt,'r') as f:
    strings = f.read()
    strings = strings.lower()
    lines = strings.split('\n')
    f.close()

In [5]:
classes = lines[0].split(',')
del lines[0]

In [6]:
df_names = list()
for a in lines:
  temp = ''
  for char in a:
    if char == ':':
      break
    else:
      temp += char
  df_names.append(temp)
del df_names[-1]

In [7]:
df_names.append('outcome')

In [8]:
df.columns = df_names

In [ ]:
df.head()


In [ ]:
#There is no use for clearning the data excessively as all datapoints are present
df.isnull().sum()

In [ ]:
#Data outcome (inbalanced)
spreading = Counter(df['outcome'])
spreading

In [ ]:
#There is a separate testing set, thus the whole set is used for training purposes


In [9]:
#Using SVC once again due to the large dimension and complex relations
#Might consider reuploading the binary classification using random forect instead for practice
#due to the nature of the dataset, several columns for x needs to be one-hot encoded
#the outcome can be categorically encoded :)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.model_selection import cross_validate

In [10]:
copy = df.copy()
y = copy.pop('outcome')

In [11]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [12]:
outcome_dict = dict()
check = df['outcome']
i = 0
for a in y:
  outcome_dict[a] = check[i]
  i += 1

In [ ]:
df.dtypes

In [13]:
copy = df.copy()
copy.pop('outcome')

0          normal.
1          normal.
2          normal.
3          normal.
4          normal.
            ...   
4898426    normal.
4898427    normal.
4898428    normal.
4898429    normal.
4898430    normal.
Name: outcome, Length: 4898431, dtype: object

In [14]:
copy = pd.get_dummies(copy)

In [ ]:
clf_svm = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf = OneVsRestClassifier(clf_svm).fit(copy,y)

In [ ]:
path_gz_test = get_file('kddcup.testdata.gz', origin='http://kdd.ics.uci.edu/databases/kddcup99/kddcup.testdata.unlabeled.gz')
corrected = get_file('corrected.gz', origin='http://kdd.ics.uci.edu/databases/kddcup99/corrected.gz')

In [ ]:
x_test = pd.read_csv(path_gz_test,header=None)
df_names.remove('outcome')
x_test.columns = df_names
x_test = pd.get_dummies(x_test)

In [ ]:
df_right = pd.read_csv(corrected,header=None)
df_names.append('outcome')
df_right.columns=df_names
y_test = df_right.pop('outcome')

In [ ]:
y_pred = clf.predict(x_test)
from sklearn import metrics
print(metrics.classification_report(y_test,y_pred,digits=3))
